### 載入套件 和定義函式

dcard 標籤要多注意，因為很長被更換

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
import urllib.request as req
import numpy as np
from scipy.linalg import norm
import pandas as pd

def tfidf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(list(s))
    # 將字中間加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 轉化為TF矩陣
    cv = TfidfVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    # 計算TF係數
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

def movestopwords(sentence):
    stopwords = stopwordslist('stopword.txt')  # 這裏加載停用詞的路徑
    outstr = ''
    for word in sentence:           
        if word not in stopwords:  
            if word != '\t'and'\n':
                outstr += word
    return outstr

def getWord(url):
    # 建立一個Request 物件，附加Request Headers 的資訊
    request = req.Request(url, headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    # 「解析」原始碼，取得每篇文章的問題
    # utf-8(比較省空間)有部分的漢字不能轉換所以要用GB18030編碼

    import bs4
    # 讓beautifulSoup協助我們解析HTML格式文件
    root = bs4.BeautifulSoup(data, "html.parser")
    titles = root.find("div", class_="sc-ba53eaa8-0 iSPQdL")  # 用列表顯示全部爬蟲下來的標題

    for title in titles:
        result = title.text.strip().replace('\n', '').replace(' ', '')
    return result
    # titles代表div標籤
    # 尋找class = "title" 的div 標籤，因為class是保留字，所以寫成class_
    # root 代表整個網頁、title是網頁標籤也是網頁標題
    # cls 是清空終端機(terminal)
    # mode = "a"是以附加的方式打開並寫入文件，因為mode = "w"會將檔案清空在寫入，mode="a"不會清空



### 抓取檔案

In [43]:
train=pd.read_csv('done_2021-10to11.csv')
# article=getWord('https://www.dcard.tw/f/relationship/p/238632575')
# article=getWord('https://www.dcard.tw/f/talk/p/239984330')
article=getWord('https://www.dcard.tw/f/talk/p/239983442')

In [7]:
train

,singer,name,lyrics,moodCat
0,魏昌澤,陰天,"['天空黑暗', '烏雲出來', '雨滴總是要落不落下來', '風中吹散', '一片花海',...",哀
1,非心聞,贈,"['一個房間，隔開我與這個世界。', '而你，從陽台俯瞰這條街。', '伴著音樂，我給你寫下...",哀
2,神隊友,神隊友,"['企業女子壘球隊群星', '石靚蘋 & 陳安心 & 楊淑媛 & 沈筱琳 & 余佳玲 & 林...",喜
3,TΔΞ泰泫,Lose My Mind,"['Lose my mind', '就算心都碎了 但是我還有妳啊', '不需要三個願望 只求...",哀
4,Black MIC,ODB,"['最近思緒太亂 悲情字句氾濫', '將我理性癱瘓 酒我直接裁罐', '喝的太快但我醉的太慢...",喜
...,...,...,...,...
1758,金文特,痛苦的解脫,"['為何一個人的安樂？', '會是另一個人的苦澀？', '捨不得 看妳苦著', '但心裡明白...",哀
1759,醬呱家族,哞哞喝牛奶,"['起床了 小朋友', '早餐準備好囉', '耶 早安', '你要你要喝什麼', '到底什麼...",喜
1760,OK Music,聽,"['海嘯般的情緒 把我吞噬 淹沒', '眼睛閉上 用心專注 的 我聽著', '你話語像沈穩的...",愛
1761,梁美琪,舞會,"['兩人的溝通有如學不上手 的 TANGO', '話總是說一半 讓我進退兩難', '總是...",愛


### 計算配適度

In [44]:
lyrics=train['lyrics']
i=0
num=0
highpri=0
for text in lyrics:
    text=movestopwords(text)
    text=text.replace(' ','')
    text=text.replace(',','，')
    if tfidf_similarity(text, article)>highpri:
        highpri=tfidf_similarity(text, article)
        num=i
    i+=1
print('配適度:',highpri,'作者:',train.singer.iloc[num],'歌名:',train.name.iloc[num], '情緒:',train.moodCat.iloc[num])


C:\Users\student\AppData\Local\Temp\ipykernel_14752\986002487.py:17: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))


配適度: 0 作者: 魏昌澤 歌名: 陰天 情緒: 哀
